In [1]:
%pylab

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
from scipy.optimize import curve_fit
from scipy.signal import savgol_filter

import tkinter as tk
from tkinter import filedialog
%matplotlib qt

In [3]:
def load_file_GUI(dir_string):

    root = tk.Tk()
    root.withdraw()

    file_path = filedialog.askopenfilename(initialdir = dir_string,
                                       multiple=True)
    return file_path

In [4]:
%run ../../GiantMol/Functions/1.2/data_fct_Adrien_point_by_point
matplotlib.rcParams.update({'font.size': 25})

In [5]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide

r0 = 2.5e-3 # 2.5e-3   # rayon piège Ca+
d0 = 4e-3/2            # longueur piège Ca+

Omega = 2.05e6*2*pi # 2.047e6

bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23          # écrantage piège réel GiantMol

wzLC = (2*pi*90806.9982303)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.270471


In [6]:
def data_retrieve(all_subdir,points_and_coord, condition_parameters, slash_cfg,**kwargs):

    myslashpos = slash_cfg[0]
    slashcond = slash_cfg[1]

    # determining number of elements on each repetition
    try:
        num_runs = ['Try'+str(kwargs.get('forcenumtry'))]
        num_runs_aux = [runs[myslashpos[slashcond+1]+1:] for runs in all_subdir if list(points_and_coord.keys())[0] in runs]
        num_runs_aux = list(dict.fromkeys(num_runs_aux))
        num_runs_aux = len(num_runs_aux)
        all_subdir = [x for w,x in enumerate(all_subdir) if (w-kwargs.get('forcenumtry'))%num_runs_aux==0]
    except:
        num_runs = [runs[myslashpos[slashcond+1]+1:] for runs in all_subdir if list(points_and_coord.keys())[0] in runs]
        num_runs = list(dict.fromkeys(num_runs))

    # number of repetitions
    print('> Points |',len(points_and_coord))
    print('> Simulations pour chaque point |', num_runs)
    
    data0 = [[] for i in range(len(points_and_coord))] # file path to SimuType4
    data_address = [[] for i in range(len(points_and_coord))]

    # Variables à deux coordonnées : [point, try]
    shapevar = (len(points_and_coord),len(num_runs))
    Time        = []
    Temperature = []
    
    t0 = time.clock()
    print("Hello")

    # write variables
    for k, address in enumerate(all_subdir):

    # in-loop variables
        pnt = k // len(num_runs)  # actual point
        rep = k  % len(num_runs)          # actual repetition

        # get only .dat files in each simulation directory
        onlyfiles = [f for f in listdir(address) if isfile(join(address, f)) and not "xva" in f and ".dat" in f]
        # build path file
        data0[pnt].append('{}/{}'.format(address,sort(onlyfiles)[0].strip('.dat')))
        data_address[pnt].append(address)

        # load fluorescence and T
        Time.append( loadtxt(str(data0[pnt][0])+'.dat',unpack=True)[0] )
        Temperature.append( loadtxt(str(data0[pnt][0])+'.dat',unpack=True)[4:7] )
        
        # load cloud size before injection
        if not(rep % len(num_runs)):
            print( "Point n°", pnt )
        
        print(f'{pnt:02}','-',f'{rep:02}',' > ',data0[pnt][rep])
        
        if k == len(points_and_coord)*len(num_runs) - 1:
            break

    t1 = time.clock() - t0
    print("Time elapsed: ", t1, 's') # CPU seconds elapsed (floating point)
    print("Time elapsed: ", t1/60, 'm') # CPU seconds elapsed (floating point)
    
    data_name = [data_address, data0]
    
    return data_name, num_runs, Time, Temperature 

# Extract data as npz

In [7]:
file_path = load_file_GUI('/home/adrian/Documents/Programmes/Python/GiantMol/SimAnalysisPointbyPoint')
print(file_path)

In [96]:
## GUI for data loading
# Select one data file all the way down to the directories
# SELECT Temp_SimuTypeQ_N ... .dat

file_cfg, slash_cfg, all_subdir = load_gui(filter_nocomplete=1)

file_path = file_cfg[0]
dir_path = file_cfg[1]
work_rep = file_cfg[2]
filename = file_cfg[3]

myslashpos = slash_cfg[0]
slashcond = slash_cfg[1]

> Répertoire : K:/Universite_Simus/20211201/DC11_RF19/Try00
> Filename : Temp_SimuType0_N01024_Vrf0040_Udc0.7000D+01V_nt1000.dat
> myslashpos | [2, 19, 28, 38]
> slashcond | -2
Points deleted because they were not complete []   0 pt(s)
Total number of data directories 4


In [97]:
## Condition for data
points_and_coord, condition_parameters = simu_conditions(all_subdir, myslashpos, slashcond, filename)

['DC11_RF19', 'DC11_RF20', 'DC11_RF21', 'DC11_RF22']
000 > DC11_RF19
001 > DC11_RF20
002 > DC11_RF21
003 > DC11_RF22
> condition names ['DC', 'RF']
> number of points 4
> N_ions = 1024
> e_GMol = 50


In [98]:
## Data loading (1D arrays)
data_name, num_runs, Time, Temperature \
= data_retrieve(all_subdir,points_and_coord,condition_parameters,slash_cfg) # stopping_power  ,forcenumtry=28

> Points | 4
> Simulations pour chaque point | ['Try00']
Hello


H:\Programmes\Anaconda3\envs\py307\lib\site-packages\ipykernel_launcher.py:29: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


Point n° 0
00 - 00  >  K:/Universite_Simus/20211201\DC11_RF19\Try00/Temp_SimuType0_N01024_Vrf0040_Udc0.7000D+01V_nt1000
Point n° 1
01 - 00  >  K:/Universite_Simus/20211201\DC11_RF20\Try00/Temp_SimuType0_N01024_Vrf0045_Udc0.7000D+01V_nt1000
Point n° 2
02 - 00  >  K:/Universite_Simus/20211201\DC11_RF21\Try00/Temp_SimuType0_N01024_Vrf0050_Udc0.7000D+01V_nt1000
Point n° 3
03 - 00  >  K:/Universite_Simus/20211201\DC11_RF22\Try00/Temp_SimuType0_N01024_Vrf0055_Udc0.7000D+01V_nt1000
Time elapsed:  59.25419090000014 s
Time elapsed:  0.9875698483333356 m


H:\Programmes\Anaconda3\envs\py307\lib\site-packages\ipykernel_launcher.py:58: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


In [99]:
for j,k in enumerate(data_name[0]):
    cond = k[0][29:38]
#     cond = k[0][52:61]
    print(cond)
    outfile = 'Time_and_temp_RFHEAT_'+str(cond)
    np.savez( outfile, time=Time[j], temp = mean(Temperature[j],axis=0)) # dirname_hc=re.sub(r'\\',r'/',dirname_hc)

DC11_RF19
DC11_RF20
DC11_RF21
DC11_RF22


In [13]:
# load check
cond = 'DC11_RF21'
outfile = 'Time_and_temp_RFHEAT_'+str(cond)
with load(outfile+'.npz') as data:
    a = data['time']
    b = data['temp']
print(a)
print(b)
print(shape(a))
print(shape(b))

FileNotFoundError: [Errno 2] No such file or directory: 'Time_and_temp_RFHEAT_DC11_RF21.npz'

In [81]:
whos

Variable                                       Type                          Data/Info
--------------------------------------------------------------------------------------
C_e                                            float                         1.602e-19
Omega                                          float                         12880529.879718151
Path                                           type                          <class 'pathlib.Path'>
T_max                                          int                           50
T_thr                                          int                           1
Temperature                                    list                          n=24
Time                                           list                          n=24
Urf                                            list                          n=25
a                                              ndarray                       100000: 100000 elems, type `float64`, 800000 bytes (781.25 kb

# Work with data

In [100]:
file_path = load_file_GUI('/home/adrian/Documents/Programmes/Python')
print(file_path)

('K:/Universite_Simus/20211201/Time_and_temp_RFHEAT_DC11_RF00.npz', 'K:/Universite_Simus/20211201/Time_and_temp_RFHEAT_DC11_RF01.npz', 'K:/Universite_Simus/20211201/Time_and_temp_RFHEAT_DC11_RF02.npz', 'K:/Universite_Simus/20211201/Time_and_temp_RFHEAT_DC11_RF03.npz', 'K:/Universite_Simus/20211201/Time_and_temp_RFHEAT_DC11_RF04.npz', 'K:/Universite_Simus/20211201/Time_and_temp_RFHEAT_DC11_RF05.npz', 'K:/Universite_Simus/20211201/Time_and_temp_RFHEAT_DC11_RF06.npz', 'K:/Universite_Simus/20211201/Time_and_temp_RFHEAT_DC11_RF07.npz', 'K:/Universite_Simus/20211201/Time_and_temp_RFHEAT_DC11_RF08.npz', 'K:/Universite_Simus/20211201/Time_and_temp_RFHEAT_DC11_RF11.npz', 'K:/Universite_Simus/20211201/Time_and_temp_RFHEAT_DC11_RF12.npz', 'K:/Universite_Simus/20211201/Time_and_temp_RFHEAT_DC11_RF13.npz', 'K:/Universite_Simus/20211201/Time_and_temp_RFHEAT_DC11_RF14.npz', 'K:/Universite_Simus/20211201/Time_and_temp_RFHEAT_DC11_RF15.npz', 'K:/Universite_Simus/20211201/Time_and_temp_RFHEAT_DC11_RF16.

In [103]:
# load check
# cond = 'DC11_RF00'
# outfile = 'Time_and_temp_RFHEAT_'+str(cond)
Time = []
Temperature = []
for j,k in enumerate(file_path):
    with load(k) as data:
        Time.append( data['time'] )
        Temperature.append( data['temp'] )
    print(j,k)
print(shape(Time))
print(shape(Temperature))

0 K:/Universite_Simus/20211201/Time_and_temp_RFHEAT_DC11_RF00.npz
1 K:/Universite_Simus/20211201/Time_and_temp_RFHEAT_DC11_RF01.npz
2 K:/Universite_Simus/20211201/Time_and_temp_RFHEAT_DC11_RF02.npz
3 K:/Universite_Simus/20211201/Time_and_temp_RFHEAT_DC11_RF03.npz
4 K:/Universite_Simus/20211201/Time_and_temp_RFHEAT_DC11_RF04.npz
5 K:/Universite_Simus/20211201/Time_and_temp_RFHEAT_DC11_RF05.npz
6 K:/Universite_Simus/20211201/Time_and_temp_RFHEAT_DC11_RF06.npz
7 K:/Universite_Simus/20211201/Time_and_temp_RFHEAT_DC11_RF07.npz
8 K:/Universite_Simus/20211201/Time_and_temp_RFHEAT_DC11_RF08.npz
9 K:/Universite_Simus/20211201/Time_and_temp_RFHEAT_DC11_RF11.npz
10 K:/Universite_Simus/20211201/Time_and_temp_RFHEAT_DC11_RF12.npz
11 K:/Universite_Simus/20211201/Time_and_temp_RFHEAT_DC11_RF13.npz
12 K:/Universite_Simus/20211201/Time_and_temp_RFHEAT_DC11_RF14.npz
13 K:/Universite_Simus/20211201/Time_and_temp_RFHEAT_DC11_RF15.npz
14 K:/Universite_Simus/20211201/Time_and_temp_RFHEAT_DC11_RF16.npz
15 K:

In [116]:
Urf = list(linspace(60,68,9)) \
    + [62.2, 62.4, 62.6, 62.8, 63.2, 63.4, 63.6, 63.8] \
    + list(linspace(40,55,4)) \
    + list(linspace(70,85,4))
print(Urf)
print(len(Urf))

qx = array(Urf)*4*C_e/(m_Ca*r0**2*Omega**2)
print(qx)

[60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 62.2, 62.4, 62.6, 62.8, 63.2, 63.4, 63.6, 63.8, 40.0, 45.0, 50.0, 55.0, 70.0, 75.0, 80.0, 85.0]
25
[0.55714841 0.56643422 0.57572002 0.58500583 0.59429164 0.60357744
 0.61286325 0.62214906 0.63143486 0.57757718 0.57943435 0.58129151
 0.58314867 0.58686299 0.58872015 0.59057731 0.59243448 0.37143227
 0.41786131 0.46429034 0.51071938 0.65000648 0.69643551 0.74286455
 0.78929358]


In [251]:
imax = len(Urf)
print('imax',imax)

T_max = 50
T_thr = 1

figure(10); clf();
i = 15
print(i,Urf[i])
#     subplot(1,9,i+1)
semilogy(Time[i]*1e3,savgol_filter(Temperature[i],31,1),label=' Vrf = '+str(Urf[i]))
# plot(Time[i][:-1]*1e3,diff( savgol_filter(Temperature[i],31,1)),label=' Vrf = '+str(Urf[i]))
# semilogy(Time[i]*1e3,func2(Time[i]*1e3,4,1,0.8,1e-10,0.46),label='fit')


p0  = array([x0, A0, B0,  C0, D0])
#     semilogy(time[i][:]*1e3,Temp[i][:],label=' Vrf = '+str(Urf[i]))
#         semilogy(time[k_thr],Temp[i,j,k_thr],'o')
grid()
legend(fontsize=12)

imax 25
15 63.6


In [228]:
len(Time[i])

10673

In [106]:
# plot all temperature curves

imax = len(Urf)
print('imax',imax)

T_max = 50
T_thr = 1

figure(1); clf();
for i in range(0,imax-1):
    print(i)
    k_max = argmin(abs(Temperature[i] - T_max))
    k_thr = argmin(abs(Temperature[i] - T_thr))
    k_min = k_thr - (k_max - k_thr)
    if k_min < 0:
        k_min = 0

    i_aux = where(diff(abs(diff(Temperature[i][k_thr:k_max])))>2)
    if size(i_aux) != 0 :
        if (i_aux[0][0]+k_thr < k_max):
            k_max = i_aux[0][0]+k_thr

#     subplot(1,9,i+1)
    subplot(imax//3,4,i+1)
    semilogy(Time[i][k_min:k_max]*1e3,Temperature[i][k_min:k_max],label=' Vrf = '+str(Urf[i]))
#     semilogy(time[i][:]*1e3,Temp[i][:],label=' Vrf = '+str(Urf[i]))
#         semilogy(time[k_thr],Temp[i,j,k_thr],'o')
    grid()
    legend(fontsize=12)
tight_layout()

imax 25
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23


In [206]:
# Functions for fit

def func2(t, t0, A, B,  C, D):
    return A*(t-t0) / (C+(t-t0)**2)**D + B

def func2_Dfix(t, t0, A, B,  C):
    return A*(t-t0) / (C+(t-t0)**2)**0.441678 + B

def func2_simple(t, t0, A, B):
    return A*(t-t0) / ((t-t0)**2)**0.441678 + B # Davg = 0.4416784564098122

def dfunc2_dt(t, t0, A, B,  C, D):
    return 10**(A*(C + (t - t0)**2)**(-D)*(t - t0) + B)*(-A*D*(C + (t - t0)**2)**(-D)*(t - t0)*(2*t - 2*t0)/(C + (t - t0)**2) + A*(C + (t - t0)**2)**(-D))*log(10)

In [205]:
# Fit all temperature curves with above function
# savgol_filter used to filter a little bit
# smooth the temperatures that oscillates a bit too much

T_max = 85
T_thr = 1
fit_param = zeros((imax,5))
fit_pcov  = zeros((imax,5,5))
my_order = []

figure(1); clf()
for i in range(0,imax):
    print('<<<',i,'>>>')
    k_max = argmin(abs(Temperature[i] - T_max))
    k_thr = argmin(abs(Temperature[i] - T_thr))

    i_aux = where(diff(abs(diff(Temperature[i][k_thr:k_max])))>2)
    if size(i_aux) != 0 :
        if (i_aux[0][0]+k_thr < k_max):
            k_max = i_aux[0][0]+k_thr

    k_min = k_thr - int(2*(k_max - k_thr))
    if k_min < 0: k_min = 0
#         print(k_min)

    x  = Time[i][k_min:k_max]
    print(Urf[i])
    print(len(Temperature[i]),k_max,k_thr,k_min)
    try:
        y  = savgol_filter(Temperature[i][k_min:k_max],31,1)
    except:
        y  = Temperature[i][k_min:k_max]
        print('no filter')
        
    x0 = Time[i][k_thr]

    A0  = 1e3
    B0  = log10(Temperature[i][k_thr])
    C0  = 1.0e-8
    D0  = 0.5
#     p0  = array([x0, A0, B0,  C0, D0])
    p0  = array([x0, A0, B0, C0])
    
    my_order.append(i)

    try:
        popt, pcov = curve_fit(func2, x, log10(y), p0, maxfev= 50000)
        fit_param[i,:] = popt[0], popt[1], popt[2], popt[3], popt[4]
        fit_pcov[i,:]  = pcov

        y_fit = func2(x,popt[0], popt[1], popt[2], popt[3], popt[4])
        y_fit = 10**y_fit

        subplot(6,4,i+1)
        plot(x*1e3, y    ,label=' Vrf = '+str(Urf[i]))
        plot(x*1e3, y_fit,label='Fit',ls='--')
        grid()
        legend(fontsize=10)
        # select cases
        # non zero fit value
    except TypeError:
        print('TypeError for ',Urf[i])
    except ValueError:
        print('ValueError for ',Urf[i])
tight_layout()

<<< 0 >>>
60.0
31892 31732 30753 28795
TypeError for  60.0
<<< 1 >>>
61.0
23609 21381 20630 19128
TypeError for  61.0
<<< 2 >>>
62.0
13529 12618 12171 11277
TypeError for  62.0
<<< 3 >>>
63.0
8329 5953 5630 4984
TypeError for  63.0
<<< 4 >>>
64.0
5901 5898 5300 4104
TypeError for  64.0
<<< 5 >>>
65.0
6127 4549 3861 2485
TypeError for  65.0
<<< 6 >>>
66.0
3489 3475 2948 1894
TypeError for  66.0
<<< 7 >>>
67.0
4980 2467 2130 1456
TypeError for  67.0
<<< 8 >>>
68.0
4388 1399 974 124
TypeError for  68.0
<<< 9 >>>
62.2
13609 11524 10515 8497
TypeError for  62.2
<<< 10 >>>
62.4
9435 6977 6176 4574
TypeError for  62.4
<<< 11 >>>
62.6
13522 10221 9333 7557
TypeError for  62.6
<<< 12 >>>
62.8
11919 8457 7843 6615
TypeError for  62.8
<<< 13 >>>
63.2
10089 8804 7787 5753
TypeError for  63.2
<<< 14 >>>
63.4
9969 7276 6310 4378
TypeError for  63.4
<<< 15 >>>
63.6
10673 6787 6125 4801
TypeError for  63.6
<<< 16 >>>
63.8
9071 6734 6090 4802
TypeError for  63.8
<<< 17 >>>
40.0
100000 372 372 372
no fi

In [134]:
# select only Urf values
# for which non zero fit
# and sort ascending Urf
Y = Urf
X = my_order
my_order  = [x for _,x in sorted(zip(Y,X))]
Urf_order = [x for x,_ in sorted(zip(Y,X))]
qx_order  = [x for _,x in sorted(zip(Y,qx))]
# delete manually
# other innapropriate values
to_delete = [17,18,19,22,23,24]
for i,j in enumerate(to_delete):
    my_order.remove(j)
    Urf_order.remove(Urf[i])
    qx_order.remove(qx[i])

print(Urf_order)
print(my_order)
print(qx_order)

# resort all variables
# in order so
# Urf ascending
imax_order = len(Urf_order)
fit_param_order = zeros((imax_order,5))
fit_pcov_order  = zeros((imax_order,5,5))
Time_order = []
Temperature_order = []
for i,j in enumerate(my_order):
    fit_param_order[i,:] = fit_param[j,:]
    fit_pcov_order[i,:]  = fit_pcov[j,:]
    Time_order.append(Time[j])
    Temperature_order.append(Temperature[j])

[40.0, 45.0, 50.0, 55.0, 62.2, 62.4, 62.6, 62.8, 63.2, 63.4, 63.6, 63.8, 66.0, 67.0, 68.0, 70.0, 75.0, 80.0, 85.0]
[5, 6, 7, 8, 11, 12, 3, 13, 20, 14, 15, 16, 4, 0, 1, 2, 9, 10, 21]
[0.3714322727409249, 0.41786130683354045, 0.46429034092615606, 0.5107193750187717, 0.5775771841121381, 0.5794343454758427, 0.5812915068395474, 0.5831486682032521, 0.5868629909306613, 0.5887201522943659, 0.5905773136580705, 0.5924344750217752, 0.6128632500225261, 0.6221490568410492, 0.6314348636595722, 0.6500064772966185, 0.6964355113892341, 0.7428645454818498, 0.7892935795744653]


In [126]:
# draw one temp and fit
figure(3); clf()
cm = pylab.get_cmap('jet')

i = 0

print(Urf_order[i])
# print(fit_pcov_order[i,:])
y_fit = func2(Time_order[i],
              fit_param_order[i,0],
              fit_param_order[i,1],
              fit_param_order[i,2],
              fit_param_order[i,3],
              fit_param_order[i,4])
y_fit = 10**y_fit
subplot(121)
plot((Time_order[i])*1e3,Temperature_order[i],color='r')
plot((Time_order[i])*1e3,y_fit,'--',color='k')
xlabel('time [ms]')
ylabel('T [K]')
grid()

# xlim(3.6,3.7)
# ylim(-1,10)
subplot(122)
semilogy((Time_order[i])*1e3,savgol_filter(Temperature_order[i],31,1),color='r')
semilogy((Time_order[i])*1e3,y_fit,'--',color='k')
xlabel('time [ms]')
grid()
tight_layout()

40.0


In [135]:
# draw fit parameters
# as a function of Urf
str_labels = ['t0', 'A', 'B', 'C', 'D']
figure('Param'); clf();
for i in range(0,5):
    subplot(1,5,i+1)
    if i == 2:
        plot(qx_order,10**(fit_param_order[:,i]),'o-', label=str_labels[i])
    else:
        plot(qx_order,(fit_param_order[:,i]),'o-', label=str_labels[i])
    xlabel('Urf')
    grid()
    legend()

In [340]:
# Draw Heating rate H
# as a function of temperature T
xa = -2
xb = 50
ya = 6e-6
yb = 1e0
figure(29,clear='True')
ax = subplot(111)
for i in [0,1,13,17]: #  range(0,imax_order)
    t0, A, B, C, D = fit_param_order[i,:]
    y  = func2(Time_order[i],t0, A, B, C, D);
    y  = 10**y
    dy = dfunc2_dt(Time_order[i], t0, A, B,  C, D)
#     ax.semilogy((Time_order[i]-t0)*1e3,dy*1e-6,label=str(Urf[i]),color=cm(i/imax_order))
    ax.semilogy(savgol_filter(Temperature_order[i],31,1),dy*1e-6,label=str(Urf_order[i]),color=cm(i/imax_order))
#     ax.semilogy(Temperature_order[i][:-1],diff(Temperature_order[i])*1e-6,label=str(Urf_order[i]),color=cm(i/imax_order))
    ax.set_xlabel('T [K]')
    ax.set_ylabel('Heating rate [K/µs]')
#     scatter(t0,4e5,color=cm(i/imax),marker='+')
#     semilogy(y,dy*1e-3,label=str(Urf[i]),color=cm(i/imax))
ax.set_xlim([xa,xb])
ax.set_ylim([ya,yb])
        
# locmaj = matplotlib.ticker.LogLocator(base=10,numticks=12) 
# ax.yaxis.set_major_locator(locmaj)
# locmin = matplotlib.ticker.LogLocator(base=10.0,subs=(0.2,0.4,0.6,0.8),numticks=12)
# ax.yaxis.set_minor_locator(locmin)
# ax.yaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())
# ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.1))
ax.legend(title=r'$U_{RF}$',ncol=2)
ax.grid(b=True, which='major', linestyle='-')
ax.grid(b=True, which='minor', linestyle='--')
tight_layout()

In [500]:
# Draw Heating rate H
# as a function of temperature T
xa = -2
xb = 50
ya = 6e-6
yb = 1e0
coucou = []
figure(31,clear='True')
ax = subplot(111)
for i in [0,1,13,17]: #  range(0,imax_order)
    t0, A, B, C, D = fit_param_order[i,:]
    x  = savgol_filter(Temperature_order[i],31,1)
    y  = func2(Time_order[i],t0, A, B, C, D);
    y  = 10**y
    dy = dfunc2_dt(Time_order[i], t0, A, B,  C, D) /x
#     ax.semilogy((Time_order[i]-t0)*1e3,dy*1e-6,label=str(Urf[i]),color=cm(i/imax_order))
    ax.semilogy(x,dy*1e-6,label=str(Urf_order[i]),color=cm(i/imax_order),marker='+')
#     ax.semilogy(Temperature_order[i][:-1],diff(Temperature_order[i])*1e-6,label=str(Urf_order[i]),color=cm(i/imax_order))
    coucou.append( argmin(abs(x-0.5)) )

#     ax.vlines(x[coucou],1e-5,0.5e0,color=cm(i/imax_order))
    ax.set_xlabel('T [K]')
    ax.set_ylabel('Heating rate / T [/µs]')
#     scatter(t0,4e5,color=cm(i/imax),marker='+')
#     semilogy(y,dy*1e-3,label=str(Urf[i]),color=cm(i/imax))
ax.set_xlim([xa,xb])
ax.set_ylim([ya,yb])
        
# locmaj = matplotlib.ticker.LogLocator(base=10,numticks=12) 
# ax.yaxis.set_major_locator(locmaj)
# locmin = matplotlib.ticker.LogLocator(base=10.0,subs=(0.2,0.4,0.6,0.8),numticks=12)
# ax.yaxis.set_minor_locator(locmin)
# ax.yaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())
# ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.1))
ax.legend(title=r'$U_{RF}$',ncol=2)
ax.grid(b=True, which='major', linestyle='-')
ax.grid(b=True, which='minor', linestyle='--')
tight_layout()

In [508]:
X = []
Y = []
for i,j in enumerate([0,1,13,17]):
#     x = savgol_filter(Temperature_order[i],31,1)
    x  = func2(Time_order[i],t0, A, B, C, D)
    X.append(Urf_order[j])
    Y.append(x[coucou[i]])
print(X,Y)
figure('test',clear='True')
plot(X,Y)

[55.0, 60.0, 65.0, 70.0] [3.816836351572443, 2.82087657947867, 2.031329563329401, -0.43716055296305467]


In [489]:
110/3493 * 15, x[15]

(0.47237331806470084, 0.00586806794642175)

# t0 and T as a function of Urf

In [447]:
t0    = zeros(imax_order)
y_t0  = zeros(imax_order)
dy_t0 = zeros(imax_order)
for i in range(0, imax_order):
    t0[i], A, B, C, D = fit_param_order[i,:]
    y  = func2(t0[i],t0[i], A, B, C, D);
    y_t0[i]  = 10**y
    dy_t0[i] = dfunc2_dt(t0[i], t0[i], A, B,  C, D)

In [346]:
def my_lin(x,a,b):
    return a*x+b
def my_power_law(x,a,b):
    return a*(x**-b)

In [446]:
# log plot
figure(321,clear='True')
ax = subplot(111)
ax.loglog(Urf_order,t0*1e3,ls='-',marker='+',mew=5,ms=10)
ax.set_xlabel('$U_{rf}$ [V]')
ax.set_ylabel('$t_0$ [ms]') # '$T(t_0) / t_0$ [K/s]'
ax.axis([5*10, 72, 1, 200])
ax.grid(b=True, which='major', linestyle='-')
ax.grid(b=True, which='minor', linestyle='--')

tight_layout()

In [450]:
figure(322); clf();
ax = subplot(111)
ax.plot(Urf_order,y_t0,ls='-',marker='+',mew=5,ms=10)
ax.grid()
ax.set_xlabel('$U_{rf}$ [V]')
ax.set_ylabel('$T(t_0)$ [K]') # '$T(t_0) / t_0$ [K/s]'

popt, pcov = curve_fit(my_lin,Urf_order,y_t0)
ax.plot(Urf_order,my_lin(array(Urf_order),popt[0],popt[1]),
    color='k',ls='--')

ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=1))
ax.yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.1))

ax.text(64,0.815,f'$T(t_0) = {popt[0]:.2e} U_{{rf}} + {popt[1]:.2f}$',
            color='xkcd:black',fontsize='20',weight="bold",
            ha='left',va='center')

tight_layout()

# identique à 10**B

In [449]:
figure(323); clf();
subplot(111)
plot(Urf_order,y_t0,ls='-',marker='+',mew=5,ms=10)
grid()
xlabel('$U_{rf}$ [V]')
ylabel('$T(t_0)$ [K]') # '$T(t_0) / t_0$ [K/s]'

tight_layout()

# identique à 10**B